# Assignment 1: DepMap Data Analysis - SOLUTION

## Integrating Gene Expression and Dependency to Discover Therapeutic Targets

We want to use depmap gene effect and expression data to investigate the gene PKMYT1.
PKMYT1 is a cellcycle kinase that prevents premature Cdk1 activation. It has recently gained attention
as a potnetial cancer target.
In this project we want to find out which cancers depedn on PKMYT1,and in which cancers is PKMYT1 highly exoressed.
We will then see how expression and dependncy are correlated and if this correclation is statistically significant.
In the next stage of the analkysis we will find genes that show similar patterns of cdependncy and expression acrosss all cancer cell lines
using a correlation analysis. We will check the top 100 correlated genes and investigate them further:
How much overlap is there between expression and dependncy? What pathways is the gene collection enriched for?


**Learning Objectives:**
- Analyze relationships between gene expression and functional dependencies
- Implement correlation analyses across multiple data modalities
- Integrate multi-omic data to identify therapeutic targets
- Create publication-quality visualizations of complex genomic data

## Setup and Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.cluster import hierarchy
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print("Libraries loaded successfully!")

Libraries loaded successfully!


---

## Part 1: Data Loading and Initial Analysis (15 points)
Initially we want to know what cancer types are most dependent on PKMYT1 and in which cancers is it most highly expressed.
You 
### 1.1: Load Data Function

In [3]:
def load_depmap_data(crispr_path, expression_path):
    """
    Load DepMap CRISPR dependency and gene expression data.
    Returns:
    --------
    crispr_df : pandas.DataFrame
        CRISPR dependency scores (rows=cell lines, columns=genes)
    expression_df : pandas.DataFrame
        Gene expression values (rows=cell lines, columns=genes)
    metadata : pandas.DataFrame
        Cell line metadata (lineage, disease, etc.)
    """
    # Load data
    crispr_df = pd.read_csv(crispr_path)
    expression_df = pd.read_csv(expression_path)
    
    # Define metadata columns
    metadata_cols = ['model_id', 'cell_line_name', 'stripped_cell_line_name', 
                     'oncotree_lineage', 'oncotree_primary_disease', 
                     'oncotree_subtype', 'oncotree_code', 'ccle_name', 'depmap_model_type']
    
    # Extract metadata (same for both datasets)
    metadata = crispr_df[metadata_cols].copy()
    
    # Get gene columns (everything after metadata)
    gene_cols = [col for col in crispr_df.columns if col not in metadata_cols]
    
    # Create gene-only dataframes
    crispr_genes = crispr_df[gene_cols]
    expression_genes = expression_df[gene_cols]
    
    
    print("✓ Data loaded successfully")
    print(f"  Cell lines: {len(crispr_genes)}")
    print(f"  Genes: {len(gene_cols)}")
    print(f"  Cancer types: {metadata['oncotree_lineage'].nunique()}")
    
    return crispr_genes, expression_genes, metadata

# Load the data
crispr_data, expression_data, metadata = load_depmap_data(
    '/Users/hh65/depmap_data/matched_crispr.csv',
    '/Users/hh65/depmap_data/matched_expression.csv'
)

✓ Data loaded successfully
  Cell lines: 1091
  Genes: 17121
  Cancer types: 29


### 1.2: Summary Statistics